<a href="https://colab.research.google.com/github/chang2694/Intrusion_detection_system/blob/main/IDS_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/IDS NSL-KDD

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/IDS NSL-KDD


In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
train =  pd.read_csv('NSL_KDD_Train.csv', names = col_names)
test =  pd.read_csv('NSL_KDD_Test.csv', names = col_names)

In [ ]:
train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,146,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,232,8153,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,199,420,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
125969,8,105,145,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125970,0,2231,384,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125971,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.loc[train['label'] != 'normal', 'label'] = 'attack'
test.loc[test['label'] != 'normal', 'label'] = 'attack'

In [ ]:
obj = []
for col_names in train.columns:
  if train[col_names].dtypes == 'object':
    obj.append(col_names)
obj.pop()
obj

['protocol_type', 'service', 'flag']

In [ ]:
df_train = train.filter(obj, axis = 1)
df_test = test.filter(obj, axis = 1)

In [ ]:
oneHot_enc = preprocessing.OneHotEncoder()
df_train = oneHot_enc.fit_transform(df_train)
df_test = oneHot_enc.transform(df_test)
df_train.toarray()

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.]])

In [ ]:
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,attack
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [ ]:
label_enc = preprocessing.LabelEncoder()
label_train = label_enc.fit_transform(train['label'])
label_test = label_enc.transform(test['label'])

In [ ]:
encoded_df = pd.DataFrame(df_train.toarray(), columns=oneHot_enc.get_feature_names_out(obj))
train = train.drop(columns=obj)
train = train.drop(columns='label')
train = pd.concat([train, encoded_df], axis = 1)

encoded_df = pd.DataFrame(df_test.toarray(), columns=oneHot_enc.get_feature_names_out(obj))
test = test.drop(columns=obj)
test = test.drop(columns='label')
test = pd.concat([test, encoded_df], axis = 1)

In [ ]:
train.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,146,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,232,8153,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,199,420,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
X_train = train
y_train = label_train

X_test = test
y_test =label_test

In [ ]:
for col_names in X_train:
  if X_train[col_names].dtypes == 'object':
    print(col_names)

In [ ]:
scaler = preprocessing.Normalizer()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(100, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(100, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(100, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
y_train = pd.get_dummies(y_train)*1
y_train = y_train.values

In [ ]:
y_test = pd.get_dummies(y_test)*1
y_test = y_test.values

In [ ]:
y_train

array([[0, 1],
       [0, 1],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
3937/3937 [==============================] - 22s 4ms/step - loss: 0.0816 - accuracy: 0.9711
Epoch 2/100
3937/3937 [==============================] - 16s 4ms/step - loss: 0.0479 - accuracy: 0.9837
Epoch 3/100
3937/3937 [==============================] - 17s 4ms/step - loss: 0.0384 - accuracy: 0.9874
Epoch 4/100
3937/3937 [==============================] - 16s 4ms/step - loss: 0.0320 - accuracy: 0.9900
Epoch 5/100
3937/3937 [==============================] - 15s 4ms/step - loss: 0.0286 - accuracy: 0.9910
Epoch 6/100
3937/3937 [==============================] - 15s 4ms/step - loss: 0.0260 - accuracy: 0.9922
Epoch 7/100
3937/3937 [==============================] - 15s 4ms/step - loss: 0.0242 - accuracy: 0.9926
Epoch 8/100
3937/3937 [==============================] - 15s 4ms/step - loss: 0.0228 - accuracy: 0.9932
Epoch 9/100
3937/3937 [==============================] - 15s 4ms/step - loss: 0.0218 - accuracy: 0.9936
Epoch 10/100
3937/3937 [==============================] - 15s 4m

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

705/705 [==============================] - 2s 2ms/step - loss: 4.4238 - accuracy: 0.8222
Test accuracy: 0.8221699595451355
